## 1. 어텐션 메커니즘 
: 어텐션 메커니즘을 사용하면 인코더의 모든 타임스텝에서 계산된 은닉 상태를 활용 가능

#### 1.1 어텐션 가중치  
: 디코더는 각 은닉 상태마다 가중치를 다르게 적용하여 더 중요한 정보를 강조 ===> 파라미터와 마찬가지로 신경을 훈련하면서 함께 학습  
: 디코더가 입력 토큰 마다 중요도를 다르게 부여하는 것을 어텐션 메커니즘이라 함

#### 1.2 어텐션 메커니즘의 장점과 단점    
(장점) : 긴 텍스트를 처리할 때 정보 손실을 줄이는 데 매우 효과적  
&nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: 인코더의 모든 타임스텝에서 생성된 은닉 상태를 참고하여 더 정확한 출력을 생성  

(단점) : 모든 타임스텝에서 생성된 은닉 상태를 저장해야 하므로 메모리 사용량과 연산량이 증가  
&nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: 인코더가 처리할 수 있는 타임스텝의 최대 개수 정함 ===> 인력 텍스트 길이 제한


## 2. 트랜스포머  
: 기존의 인코더 - 디코더 구조를 유지하면서도 순환 신경망을 완전하게 제거한 특징을 가짐  
: 입력 텍스트를 한 토큰씩 처리할 필요 없이 한 번에 모두 처리 가능  
: 순환 신경망을 사용 X ===> `은닉 벡터`, `단어 벡터`, `임베딩 벡터`라는 표현을 사용

## 3. 셀프 어텐션 메커니즘  
: 인코더에 입력되는 토큰만으로 어텐션 가중치를 학습  
: 밀집층을 통과한 벡터를 `쿼리 벡터`라고 함  
: 같은 입력 텍스트를 두 번째 밀집층에 통과시켜 `키 벡터`를 생성 (쿼리 생성 밀집층과 키 생성 밀집층은 서로 다른 층)

#### 3.1 어텐션 점수 계산  
: 쿼리 벡터와 키 벡터가 생성되면 두 벡터를 서로 곱해서 `어텐션 점수`를 계산  
: 입력된 토큰이 3개라면 쿼리 벡터도 3개, 키 벡터도 3개가 생성 (각 쿼리 벡터와 키 벡터를 곱하면 총 9개의 어텐션 점수가 생성)  
: 그다음, 입력 텍스트를 또 다른 밀집층에 통과시켜 `값 벡터`를 계산

#### 3.2 멀티 헤드 어텐션  
: 셀프 어텐션 연산을 수행하는 하나의 단위를 `어텐션 헤드`라고 함  
: 트랜스포머는 여러개의 어텐션 헤드를 사용 ===> 이를 `멀티 헤드 어텐션`이라고 함  
: 각 어텐션 헤드에서는 쿼리, 키, 벡터를 생성하는 밀집층이 서로 다르게 사용  
: 어텐션 헤드들의 출력은 하나로 합쳐진 후, 밀집층을 통과하여 어텐션 층의 최종 출력이 됨 

## 4. 층 정규화  
(배치 정규화) : 주로 합성곱 신경망에서 널리 활용되며 층과 층 사이에 놓임  
&nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: 베치 단위로 평균과 분산을 계산하여 평균(0), 분산(1)이 되도록 조정 후, 다음 층으로 전달  

(층 정규화) : 각 샘플의 토큰마다 개별적으로 정규화를 수행하는 방식  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;: 샘플마다 길이가 달라도 독립적으로 정규화할 수 있어 샘플의 길이에 영향 받지 않음  

: 멀티 헤드 어텐션과 층 정규화 사이에는 `잔차 연결`이 추가  
: 잔차 연걸이 추가되면 이 신호가 멀티 헤드 어텐션 층을 거치지 않고 직접 앞쪽 층으로 전달 가능

## 5. 피드포워드 네트워크와 인코더 블록  
: 여기서 다루는 피드포워드는 트랜스포머의 인코더에서 멀티 헤드 어텐션과 층 정규화 다음에 나오는 밀집층을 말함  
: 첫 번째 밀집층은 ReLU 활성화 함수, 두 번째 층은 활성화 함수 사용 X, 그 다음 다시 드롭아웃 층이 추가  
: 이 세 개의 층을 또 다른 잔차 연결이 감싸게 됨  

* 멀티 헤드 어텐션과 피드포워드 네트워크의 연결 : 두 부분을 합친 후 마지막에 정규화를 다시 배치하면 트랜스포머 인코더 블록이 완성

## 6. 토큰 임베딩과 위치 인코딩  
: 자연어 처리에서는 모델이 입력된 문자를 이해할 수 있도록 토큰을 숫자로 변환하는 과정이 필요 ===> `임베딩`

#### 6.1 토큰 임베딩  
: 트랜스포머에서도 토큰을 고정된 크기의 실수 벡터로 변환하기 위해 임베딩 층을 사용  
: 트랜스포머가 문장의 의미를 정확히 이해하려면 위치 정보가 추가적으로 제공되어야 함

#### 6.2 위치 임베딩:  
: `위치 인코딩`은 사인 함수화 코사인 함수를 사용해 토큰의 위치에 따라 변하는 벡터를 생성 ===> 이를 단어 임베딩에 더함  
: 임베딩 벡터에서 짝수 번째 원소의 경우는 사인 함수를 적용하고, 홀수 번째 원소의 겨우는 코사인 함수를 적용 ===> 이를 원본 임베딩 벡터에 더함

## 7. 디코더 블록  
: `크로스 어텐션` : 디코더에서 받은 벡터를 쿼리로 사용하고, 인코더의 출력을 키와 값으로 사용  
: 디코더 블록에서는 크로스 어텐션 층이 인코더의 멀티 헤드 어텐션 층과 피드포워드 네트워크 사이에 배치  
: 크로스 어텐션 층의 전후에도 잔차 연결이 존재  
: 디코더 블록이 출력하는 값도 인코더 블록과 마찬가지로 동일한 크기의 은닉 벡터 ===> 여러 개의 디코더 블록을 반복해서 쌓을 수 있음

* 디코더는 자기회귀 모델의 방식을 따라 한 번에 하나의 토큰만 생성  

* 모델이 단순히 정답을 복사하는 방식으로 학습될 위험을 방지하기 위해 ===> 첫 번째 멀티 헤드 어텐션 층에서는 `마스킹` 처리를 함  
: 즉 디코더가 한 타임스텝에서 어텐션 점수를 계산할 때 현재 토큰까지만 참고하고. 이후의 토큰은 볼 수 없도록 제한하는 것  
===> 이런 이유로 디코더 블록의 멀티 헤드 어텐션 층을 `마스크드 멀티 헤드 어텐션 층`이라고 함